# INNOTECH Austin 2019: Data Cleansing Strategies

## Resources:
 * Data Cleansing

## Connect Google Drive on Colab

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/'My Drive'/Data-Cleansing-Innotech

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/
/content/drive/My Drive/Data-Cleansing-Innotech


## Initialize Packages

In [0]:
import pandas as pd